In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent
import os,sys
from sklearn.metrics import (precision_recall_curve,
                             confusion_matrix,
                             precision_score,recall_score,f1_score)
from cgan_mapper import *
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
def get_files_scan_one_cpu(dir):
    """Get the path of all the files in nested folders"""
    all_files = []
    for entry in os.scandir(dir):
        if entry.is_file():
            all_files.append(entry.path)
        elif entry.is_dir():
            all_files.extend(get_files_scan(entry.path))
    if len(all_files) > 0:
        return all_files
    else:
        raise FileNotFoundError("No Files found. Please check your directory.")

def scan_files_in_folder(folder):
    """Scans files in a folder and returns a list of file paths."""
    files = []
    for entry in os.scandir(folder):
        if entry.is_file():
            files.append(entry.path)
        elif entry.is_dir():
            files.extend(scan_files_in_folder(entry.path))
    return files

def get_files_scan(dir,max_workers=1):
    """Get the path of all the files in nested folders using concurrent.futures."""
    all_files = []
    if max_workers==1:
        return get_files_scan_one_cpu(dir)
    else:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_dir = {executor.submit(scan_files_in_folder, entry.path): entry.path for entry in os.scandir(dir)}
            for future in concurrent.futures.as_completed(future_to_dir):
                folder = future_to_dir[future]
                try:
                    all_files.extend(future.result())
                except Exception as e:
                    print(f"An error occurred while scanning {folder}: {e}")

        if len(all_files) > 0:
            return all_files
        else:
            raise FileNotFoundError("No Files found. Please check your directory.")

In [7]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_10"
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_8/"
ABC_JNC_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_0/"


CC3D_CG_PATH="/mnt/isilon/mkhan/complexgen/cc3d/ComplexGen/"
ABC_CG_PATH="/mnt/isilon/mkhan/complexgen/abc/ComplexGen/"

# CC3D - Boundary (Model Trained on CC3D and Tested on CC3D)

In [8]:
all_files_cc3d=root_dict(CC3D_CG_PATH)

cc3d_bndry_gt_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","gt"),4)
cc3d_bndry_pred_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"),4)

In [5]:
def read_txt(path):
    # Open the file in read mode
    with open(path, 'r') as file:
        # Read all lines and store them in a list
        lines = file.readlines()

    lines=[s.strip("\n") for s in lines]
    lines=[s.split(".")[0].split("/")[-1] for s in lines]
    return lines

In [6]:
name_map_dict={}

for path in list(all_files_cc3d.keys()):
    name=path.split("/")[-1]
    for gt_file in cc3d_bndry_gt_files:
        if name in gt_file:
            temp_file="/".join(gt_file.split("/")[-3:]).split(".")[0]
            name_map_dict[path]=[os.path.join(os.path.join(CC3D_BDN_PATH,"scan"),temp_file+".ply")]
            name_map_dict[path].append(all_files_cc3d[path])
            name_map_dict[path].append([os.path.join(os.path.join(CC3D_BDN_PATH,"bndry","gt"),temp_file+".npy"),
                                        os.path.join(os.path.join(CC3D_BDN_PATH,"jnc","gt"),temp_file+".npy")])

In [7]:
# import json
# with open("name_map.json","w") as f:
#     json.dump(name_map_dict,f)

In [7]:
def read_ply_xyz(path_list):
    """
    Reads .ply files from the provided list of file paths and categorizes them based on file type.
    
    Parameters:
    - path_list (list): List of file paths to be processed.

    Returns:
    A dictionary containing categorized data:
    - 'bndry': Boundary .ply file data (if present).
    - 'jnc': Junction .ply file data (if present).
    - 'input': Input .ply file data (if present).
    """
    data={}
    for path in path_list[1]:
        if path.split(".")[-1]=="ply":
            data['bndry']=np.asarray(read_ply_file(path).points)
        elif "corner" in path.split(".")[1]:
            data['jnc']=np.asarray(read_ply_file(path).points)
    data['input']=np.asarray(read_ply_file(path_list[0]).points)

    return data

In [9]:
pred_both=voxel2scan_label_mapper(name_map_dict['/mnt/isilon/mkhan/complexgen/cc3d/ComplexGen/User Library-1 Santa Fe_Part006'])

In [10]:
pred_both['bndry']

array([0., 0., 0., ..., 0., 0., 0.])

In [11]:
pc=read_ply_file(name_map_dict['/mnt/isilon/mkhan/complexgen/cc3d/ComplexGen/User Library-1 Santa Fe_Part006'][0])

In [12]:
new_pc=o3d.io.read_point_cloud("/mnt/isilon/mkhan/complexgen/cc3d/ComplexGen/User Library-1 Santa Fe_Part006.ply_0_input.xyz")

scale=np.max(np.stack([np.asarray(new_pc.points).max(axis=0),np.abs(np.asarray(new_pc.points).min(axis=0))]),axis=0)

divider=np.max(np.stack([np.asarray(pc.points).max(axis=0),np.abs(np.asarray(pc.points).min(axis=0))]),axis=0)

In [13]:
colors = np.zeros((pred_both['bndry'].shape[0],3), dtype=np.float64)
colors[pred_both['bndry'] == 1] = [1.0, 0.0, 0.0]
pc.points=o3d.utility.Vector3dVector(pc.points*scale/divider)
pc.colors=o3d.utility.Vector3dVector(colors)

In [14]:
o3d.io.write_point_cloud("output/check.ply",pc)

True

In [15]:
precision_scores_bndry=[]
recall_scores_bndry=[]

precision_scores_jnc=[]
recall_scores_jnc=[]

In [16]:
for key,val in tqdm(name_map_dict.items()):
    try:
        pred_both=voxel2scan_label_mapper(val)
        gt_bndry=np.load(val[2][0])
        gt_jnc=np.load(val[2][1])
        precision_scores_bndry.append(precision_score(gt_bndry,pred_both['bndry']))
        recall_scores_bndry.append(recall_score(gt_bndry,pred_both['bndry']))

        precision_scores_jnc.append(precision_score(gt_jnc,pred_both['jnc']))
        recall_scores_jnc.append(recall_score(gt_jnc,pred_both['jnc']))
    except:
        pass

  0%|          | 0/4582 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [15]:
np.mean(precision_scores_bndry).round(3),np.mean(recall_scores_bndry).round(3)

(0.461, 0.056)

In [16]:
np.mean(precision_scores_jnc).round(3),np.mean(recall_scores_jnc).round(3)

(0.09, 0.013)

# ABC - Boundary (Model Trained on ABC and Tested on ABC)

In [8]:
all_files_abc=root_dict(ABC_CG_PATH)

abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","pred"),4)


name_map_dict_abc={}

for path in list(all_files_abc.keys()):
    name=path.split("/")[-1]
    for gt_file in abc_bndry_gt_files:
        if name in gt_file:
            temp_file="/".join(gt_file.split("/")[-3:]).split(".")[0]
            name_map_dict_abc[path]=[os.path.join(os.path.join(ABC_BDN_PATH,"scan"),temp_file+".ply"),
                                     os.path.join(os.path.join(ABC_JNC_PATH,"scan"),temp_file+".ply")]
            name_map_dict_abc[path].append(all_files_abc[path])
            name_map_dict_abc[path].append([os.path.join(os.path.join(ABC_BDN_PATH,"bndry","gt"),temp_file+".npy"),
                                        os.path.join(os.path.join(ABC_JNC_PATH,"jnc","gt"),temp_file+".npy")])


In [10]:
import traceback
precision_scores_bndry_abc=[]
recall_scores_bndry_abc=[]
f1_scores_bndry_abc=[]

precision_scores_jnc_abc=[]
recall_scores_jnc_abc=[]
f1_scores_jnc_abc=[]


for key,val in tqdm(name_map_dict_abc.items()):
    try:
        pred_both=voxel2scan_label_mapper(val)
        gt_bndry=np.load(val[3][0])
        gt_jnc=np.load(val[3][1])
        precision_scores_bndry_abc.append(precision_score(gt_bndry,pred_both['bndry']))
        recall_scores_bndry_abc.append(recall_score(gt_bndry,pred_both['bndry']))
        # f1_scores_bndry_abc.append(f1_score(gt_bndry,pred_both['bndry']))

        precision_scores_jnc_abc.append(precision_score(gt_jnc,pred_both['jnc']))
        recall_scores_jnc_abc.append(recall_score(gt_jnc,pred_both['jnc']))
        # f1_scores_jnc_abc.append(f1_score(gt_bndry,pred_both['jnc']))
    except Exception as e:
        #print(traceback.print_exec())
        pass

  0%|          | 0/772 [00:00<?, ?it/s]

RPly: Unable to open file


[Open3D WARNING] Read PLY failed: unable to open file: /mnt/isilon/mkhan/scan2brep/test/BRepJd/version_0/scan/abc_0021_obj_v00/00217837/00217837_11d0df34b2f01abaaa985e5c_trimesh_004.ply


In [11]:
pred_both['jnc'].sum()

65.0

In [10]:
print(f"Recall {np.mean(recall_scores_bndry_abc).round(3)}")
print(f"Precision {np.mean(precision_scores_bndry_abc).round(3)}")

Recall 0.102
Precision 0.332


In [12]:
print(f"Recall {np.mean(recall_scores_jnc_abc).round(3)}")
print(f"Precision {np.mean(precision_scores_jnc_abc).round(3)}")

Recall 0.06
Precision 0.298


In [12]:
np.mean(recall_scores_jnc_abc), np.mean(precision_scores_jnc_abc).round(3).round(3),np.mean(f1_scores_jnc_abc)

(0.044938055316144185, 0.253, nan)

# Visualization

In [1]:
def create_pcd_with_colors(array,label,translate=[0,0,0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(array)
    colors = np.ones((array.shape[0],3), dtype=np.float64)*0.5
    if label is not None:
        colors[label == 1] = [1.0, 0.0, 0.0]
    # colors[label == 0] = [106/255, 115/255, 108/255]
    pcd.colors = o3d.utility.Vector3dVector(colors)

    return pcd.translate(np.array(translate))

def create_pcd_with_two_colors(array,label1,label2,translate=[0,0,0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(array)
    colors = np.ones((array.shape[0],3), dtype=np.float64)*0.5
    colors[label1 == 1] = [1.0, 0.0, 0.0]
    colors[label2 == 1] = [0.0, 1.0, 0.0]
    # colors[label == 0] = [106/255, 115/255, 108/255]
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd.translate(np.array(translate))


def create_pcd_with_green_colors(array,label,translate=[0,0,0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(array)
    colors = np.ones((array.shape[0],3), dtype=np.float64)*0.5
    if label is not None:
        colors[label == 1] = [0.0, 1.0, 0.0]
    # colors[label == 0] = [106/255, 115/255, 108/255]
    pcd.colors = o3d.utility.Vector3dVector(colors)

    return pcd.translate(np.array(translate))

In [2]:
BNET_JNC_ABC="/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_0"
PNET_BNDRY_ABC="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_0/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

In [3]:
def point_label_mapper(point,label,target):
    new_label=np.zeros(target.shape[0])
    nn = pynanoflann.KDTree(n_neighbors=1, metric='L1', radius=100)
    # scale=np.max(np.stack([np.asarray(target).max(axis=0),
    #                        np.abs(np.asarray(target).min(axis=0))]),axis=0)
    
    # divider=np.max(np.stack([np.asarray(point).max(axis=0),
    #                         np.abs(np.asarray(point).min(axis=0))]),axis=0)
    point=point[np.where(label==1)[0]]
    nn.fit(target)
    new_label[np.unique([nn.kneighbors(point)[1].reshape(-1)])]=1
    return new_label

def point_label_mapper_1(point,label,target):
    new_label=np.zeros(target.shape[0])
    nn = pynanoflann.KDTree(n_neighbors=1, metric='L1', radius=100)
    # scale=np.max(np.stack([np.asarray(target).max(axis=0),
    #                        np.abs(np.asarray(target).min(axis=0))]),axis=0)
    
    # divider=np.max(np.stack([np.asarray(point).max(axis=0),
    #                         np.abs(np.asarray(point).min(axis=0))]),axis=0)
    # point=point[np.where(label==1)[0]]
    nn.fit(target)
    new_label[np.unique([nn.kneighbors(point)[1].reshape(-1)])]=1
    return new_label


In [4]:
output_path="/mnt/isilon/mkhan/scan2brep/result/abc"
i=0
for key,val in tqdm(name_map_dict_abc.items()):
    # original scan
    try:
        mesh=o3d.io.read_triangle_mesh(os.path.join(SCAN_PATH,
                                                    "/".join(val[3][1].split("/")[-3:])).strip(".npy")+".obj")
        scale_orig_scan=max(np.abs(np.asarray(mesh.vertices).max()),
                            np.abs(np.asarray(mesh.vertices).min()))

        orig_scan=np.asarray(mesh.vertices)/scale_orig_scan


        # Complexgen results
        pred_both=voxel2scan_label_mapper(val) # Contains both jnc and boundary results

        # BrepDetNet Results
        bdn_pred_bndry=(np.load(val[3][0].replace("gt","pred"))>0.56).astype(int) # BrepDetNet Boundary
        scan=np.asarray(read_ply_file(val[0]).points) # This is the sampled scan (10k)

        # Junction has different points (4k)
        bdn_pred_jnc=np.load(os.path.join(BNET_JNC_ABC,"jnc",
                                        "pred","/".join(val[3][1].split("/")[-3:]))) # BrepDetNet Junction
        bdn_gt_jnc=np.load(os.path.join(BNET_JNC_ABC,"jnc",
                                        "gt","/".join(val[3][1].split("/")[-3:]))) # BrepDetNet Junction
        bdn_pred_jnc=(bdn_pred_jnc>0.55).astype(int)
        jnc_scan=np.asarray(read_ply_file(os.path.join(BNET_JNC_ABC,
                                                    "scan","/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply").points)

        
        #bdn_pred_jnc=point_label_mapper(jnc_scan,bdn_pred_jnc,scan) # Junction label mapped to sampled scan points

        # Ground Truth Boundary and Junction label
        gt_bndry=np.load(val[3][0])
        #gt_jnc=np.load(val[2][1])

        # PieNet Results
        pienet_bndry=(np.load(os.path.join(PNET_BNDRY_ABC,"bndry",
                                        "pred","/".join(val[3][1].split("/")[-3:])))>0.65).astype(int)
        
        pienet_scan=np.asarray(read_ply_file(os.path.join(PNET_BNDRY_ABC,"scan",\
                                        "/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply").points)
        
        pienet_bndry=point_label_mapper(pienet_scan,pienet_bndry,scan)

        # For boundary
        mainPcd=create_pcd_with_colors(orig_scan,None)+\
            create_pcd_with_colors(scan,gt_bndry,[3.0,0.0,0.0])+\
            create_pcd_with_colors(scan,pred_both['bndry'],[6.0,0.0,0.0])+\
            create_pcd_with_colors(scan,bdn_pred_bndry,[9.0,0.0,0.0]) +\
            create_pcd_with_colors(scan,pienet_bndry,[12.0,0.0,0.0]) +\
            create_pcd_with_green_colors(jnc_scan,bdn_gt_jnc,[15.0,0.0,0.0])+\
            create_pcd_with_green_colors(jnc_scan,pred_both['jnc'],[18.0,0.0,0.0]) +\
            create_pcd_with_green_colors(jnc_scan,bdn_pred_jnc,[21.0,0.0,0.0])

        # create_pcd_with_colors(scan,pred_both['bndry'],[6.0,0.0,0.0])+\
            # create_pcd_with_colors(scan,bdn_pred_bndry,[9.0,0.0,0.0]) +\
            # create_pcd_with_colors(scan,pienet_bndry,[12.0,0.0,0.0]) +\
        o3d.io.write_point_cloud(os.path.join(output_path,"bndry",key.split("/")[-1]+".ply"),mainPcd)
        #o3d.io.write_point_cloud("check.ply",mainPcd)

        # # For Junction
        # mainPcd_jnc=create_pcd_with_colors(scan,gt_jnc)+\
        #     create_pcd_with_colors(scan,pred_both['jnc'],[3.0,0.0,0.0])+\
        # create_pcd_with_colors(scan,bdn_pred_jnc,[6.0,0.0,0.0])

        # o3d.io.write_point_cloud("check_jnc.ply",mainPcd_jnc)

        # i+=1
        # if i==4:
        #     break
    except Exception as e:
        print(traceback.print_exc())


NameError: name 'tqdm' is not defined

# Model trained on ABC and tested on CC3D

In [9]:
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_9"
# ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_8/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"


ABL_PPL_BNDRY="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_0/"

BNET_JNC_ABC="/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_0"
PNET_BNDRY_ABC="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_0/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"
# SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

In [12]:
all_files_abc=root_dict(CC3D_CG_PATH)

abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","pred"),4)


name_map_dict_abc={}

for path in list(all_files_abc.keys()):
    name=path.split("/")[-1]
    for gt_file in abc_bndry_gt_files:
        if name in gt_file:
            temp_file="/".join(gt_file.split("/")[-3:]).split(".")[0]
            name_map_dict_abc[path]=[os.path.join(os.path.join(ABC_BDN_PATH,"scan"),temp_file+".ply"),
                                     os.path.join(os.path.join(ABC_JNC_PATH,"scan"),temp_file+".ply")]
            name_map_dict_abc[path].append(all_files_abc[path])
            name_map_dict_abc[path].append([os.path.join(os.path.join(ABC_BDN_PATH,"bndry","gt"),temp_file+".npy"),
                                        os.path.join(os.path.join(ABC_JNC_PATH,"jnc","gt"),temp_file+".npy")])


In [1]:
# import traceback
# output_path="/mnt/isilon/mkhan/scan2brep/result/abc_cc3d"
# # if not os.path.exists(output_path):
# #os.makedirs(output_path)
    
# i=0
# for key,val in tqdm(name_map_dict_abc.items()):
#     # original scan
#     try:
#         try:
#             mesh=o3d.io.read_triangle_mesh(os.path.join(SCAN_PATH,"test",
#                                                         "/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply")
#         except:
#             mesh=o3d.io.read_triangle_mesh(os.path.join(SCAN_PATH,"test_codalab",
#                                                         "/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply")
        
#         scale_orig_scan=max(np.abs(np.asarray(mesh.vertices).max()),
#                             np.abs(np.asarray(mesh.vertices).min()))

#         orig_scan=np.asarray(mesh.vertices)/scale_orig_scan

#         # Complexgen results
#         pred_both=voxel2scan_label_mapper(val) # Contains both jnc and boundary results

#         # BrepDetNet Results
#         bdn_pred_bndry=(np.load(val[3][0].replace("gt","pred"))>0.56).astype(int) # BrepDetNet Boundary
#         scan=np.asarray(read_ply_file(val[0]).points) # This is the sampled scan (10k)

#         # Junction has different points (4k)
#         bdn_pred_jnc=np.load(os.path.join(BNET_JNC_ABC,"jnc",
#                                         "pred","/".join(val[3][1].split("/")[-3:]))) # BrepDetNet Junction
#         bdn_gt_jnc=np.load(os.path.join(BNET_JNC_ABC,"jnc",
#                                         "gt","/".join(val[3][1].split("/")[-3:]))) # BrepDetNet Junction
#         bdn_pred_jnc=(bdn_pred_jnc>0.55).astype(int)
#         jnc_scan=np.asarray(read_ply_file(os.path.join(BNET_JNC_ABC,
#                                                     "scan","/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply").points)

#         #bdn_pred_jnc=point_label_mapper(jnc_scan,bdn_pred_jnc,scan) # Junction label mapped to sampled scan points

#         # Ground Truth Boundary and Junction label
#         gt_bndry=np.load(val[3][0])
#         #gt_jnc=np.load(val[2][1])

#         # PieNet Results
#         pienet_bndry=(np.load(os.path.join(PNET_BNDRY_ABC,"bndry",
#                                         "pred","/".join(val[3][1].split("/")[-3:])))>0.65).astype(int)
        
#         pienet_scan=np.asarray(read_ply_file(os.path.join(PNET_BNDRY_ABC,"scan",\
#                                         "/".join(val[3][1].split("/")[-3:])).strip(".npy")+".ply").points)
        
#         pienet_bndry=point_label_mapper(pienet_scan,pienet_bndry,scan)

#         # For boundary
#         mainPcd=create_pcd_with_colors(scan,gt_bndry)+\
#             create_pcd_with_colors(scan,pred_both['bndry'],[3.0,0.0,0.0])+\
#             create_pcd_with_colors(scan,bdn_pred_bndry,[6.0,0.0,0.0]) +\
#             create_pcd_with_colors(scan,pienet_bndry,[9.0,0.0,0.0]) +\
#             create_pcd_with_green_colors(jnc_scan,bdn_gt_jnc,[12.0,0.0,0.0])+\
#             create_pcd_with_green_colors(jnc_scan,pred_both['jnc'],[15.0,0.0,0.0]) +\
#             create_pcd_with_green_colors(jnc_scan,bdn_pred_jnc,[18.0,0.0,0.0])

#         # create_pcd_with_colors(scan,pred_both['bndry'],[6.0,0.0,0.0])+\
#             # create_pcd_with_colors(scan,bdn_pred_bndry,[9.0,0.0,0.0]) +\
#             # create_pcd_with_colors(scan,pienet_bndry,[12.0,0.0,0.0]) +\
#         o3d.io.write_point_cloud(os.path.join(output_path,key.split("/")[-1]+".ply"),mainPcd)
#         #o3d.io.write_point_cloud("check.ply",mainPcd)
#         #print(f"Successfully saved {key}")
#         # # For Junction
#         # mainPcd_jnc=create_pcd_with_colors(scan,gt_jnc)+\
#         #     create_pcd_with_colors(scan,pred_both['jnc'],[3.0,0.0,0.0])+\
#         # create_pcd_with_colors(scan,bdn_pred_jnc,[6.0,0.0,0.0])

#         # o3d.io.write_point_cloud("check_jnc.ply",mainPcd_jnc)

#         # i+=1
#         # if i==4:
#         #     break
#     except Exception as e:
#         # print(traceback.print_exc())
#         pass


## Ablation Study

##### Model Trained on ABC and tested on ABC

In [51]:
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_8/"
PPL_BDN_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_1/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

In [54]:
import traceback
output_path="/mnt/isilon/mkhan/scan2brep_abl/result/version_1_1"
# if not os.path.exists(output_path):
#     os.makedirs(output_path)
i=0
for path in tqdm(all_files_abc):
    # original scan
    try:
        name="/".join(path.split("/")[-3:]).strip(".npy")
        
        # Model with PPL 
        pred_ppl=(np.load(path)>0.56).astype(int)
        gt_bndry=np.load(os.path.join(ABC_BDN_PATH,"bndry","gt",name)+".npy")
        scan_ppl=np.asarray(read_ply_file(os.path.join(ABC_BDN_PATH,"scan",name)+".ply").points)

        # Model without PPL
        pred_wo_ppl=(np.load(os.path.join(PPL_BDN_PATH,"bndry","pred",name)+".npy")>0.56).astype(int)
        scan_wo_ppl=np.asarray(read_ply_file(os.path.join(PPL_BDN_PATH,"scan",name)+".ply").points)
        pred_wo_ppl=point_label_mapper(scan_wo_ppl,pred_wo_ppl,scan_ppl)
        
        # For boundary
        mainPcd=create_pcd_with_colors(scan_ppl,gt_bndry,[0.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_ppl,[3.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_wo_ppl,[6.0,0.0,0.0]) 

        o3d.io.write_point_cloud(os.path.join(output_path,name.split("/")[-1]+".ply"),mainPcd)
        #o3d.io.write_point_cloud("check.ply",mainPcd)

        # # For Junction
        # mainPcd_jnc=create_pcd_with_colors(scan,gt_jnc)+\
        #     create_pcd_with_colors(scan,pred_both['jnc'],[3.0,0.0,0.0])+\
        # create_pcd_with_colors(scan,bdn_pred_jnc,[6.0,0.0,0.0])

        # o3d.io.write_point_cloud("check_jnc.ply",mainPcd_jnc)
        #break
        # i+=1
        # if i==4:
        #     break
    except Exception as e:
        print(traceback.print_exc())
        break


  0%|          | 0/955 [00:00<?, ?it/s]

### Model Trained on ABC and tested on CC3D

In [57]:
ABC_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_9/"
PPL_BDN_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_3/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

all_files_abc=get_files_scan(os.path.join(ABC_BDN_PATH,"bndry","pred"))

In [60]:
import traceback
output_path="/mnt/isilon/mkhan/scan2brep_abl/result/version_1_2"
if not os.path.exists(output_path):
    os.makedirs(output_path)
i=0
for path in tqdm(all_files_abc):
    # original scan
    try:
        name="/".join(path.split("/")[-3:]).strip(".npy")
        
        # Model with PPL 
        pred_ppl=(np.load(path)>0.56).astype(int)
        gt_bndry=np.load(os.path.join(ABC_BDN_PATH,"bndry","gt",name)+".npy")
        scan_ppl=np.asarray(read_ply_file(os.path.join(ABC_BDN_PATH,"scan",name)+".ply").points)

        # Model without PPL
        pred_wo_ppl=(np.load(os.path.join(PPL_BDN_PATH,"bndry","pred",name)+".npy")>0.56).astype(int)
        scan_wo_ppl=np.asarray(read_ply_file(os.path.join(PPL_BDN_PATH,"scan",name)+".ply").points)
        pred_wo_ppl=point_label_mapper(scan_wo_ppl,pred_wo_ppl,scan_ppl)
        
        # For boundary
        mainPcd=create_pcd_with_colors(scan_ppl,gt_bndry,[0.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_ppl,[3.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_wo_ppl,[6.0,0.0,0.0]) 

        o3d.io.write_point_cloud(os.path.join(output_path,name.split("/")[-1]+".ply"),mainPcd)
        #o3d.io.write_point_cloud("check.ply",mainPcd)

        # # For Junction
        # mainPcd_jnc=create_pcd_with_colors(scan,gt_jnc)+\
        #     create_pcd_with_colors(scan,pred_both['jnc'],[3.0,0.0,0.0])+\
        # create_pcd_with_colors(scan,bdn_pred_jnc,[6.0,0.0,0.0])

        # o3d.io.write_point_cloud("check_jnc.ply",mainPcd_jnc)
        #break
        # i+=1
        # if i==4:
        #     break
    except Exception as e:
        #print(traceback.print_exc())
        #break


#### Model Trained on CC3D and Tested on CC3D

In [10]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_10/"
PPL_BDN_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_5/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

all_files_abc=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"))

In [11]:
import traceback
output_path="/mnt/isilon/mkhan/scan2brep_abl/result/version_2_2"
if not os.path.exists(output_path):
    os.makedirs(output_path)
i=0
for path in tqdm(all_files_abc):
    # original scan
    try:
        name="/".join(path.split("/")[-3:]).strip(".npy")
        
        # Model with PPL 
        pred_ppl=(np.load(path)>0.56).astype(int)
        gt_bndry=np.load(os.path.join(CC3D_BDN_PATH,"bndry","gt",name)+".npy")
        scan_ppl=np.asarray(read_ply_file(os.path.join(CC3D_BDN_PATH,"scan",name)+".ply").points)

        # Model without PPL
        pred_wo_ppl=(np.load(os.path.join(PPL_BDN_PATH,"bndry","pred",name)+".npy")>0.56).astype(int)
        scan_wo_ppl=np.asarray(read_ply_file(os.path.join(PPL_BDN_PATH,"scan",name)+".ply").points)
        pred_wo_ppl=point_label_mapper(scan_wo_ppl,pred_wo_ppl,scan_ppl)
        
        # For boundary
        mainPcd=create_pcd_with_colors(scan_ppl,gt_bndry,[0.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_ppl,[3.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_wo_ppl,[6.0,0.0,0.0]) 

        o3d.io.write_point_cloud(os.path.join(output_path,name.split("/")[-1]+".ply"),mainPcd)
        
    except Exception as e:
        #print(traceback.print_exc())
        #break
        pass


  0%|          | 0/4609 [00:00<?, ?it/s]

### Model Trained on CC3D and Tested on ABC

In [12]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_11/"
PPL_BDN_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_7/"
SCAN_PATH="/data/3d_cluster/ABC_filter/obj/"

all_files_abc=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"))

In [13]:
import traceback
output_path="/mnt/isilon/mkhan/scan2brep_abl/result/version_2_1"
if not os.path.exists(output_path):
    os.makedirs(output_path)
i=0
for path in tqdm(all_files_abc):
    # original scan
    try:
        name="/".join(path.split("/")[-3:]).strip(".npy")
        
        # Model with PPL 
        pred_ppl=(np.load(path)>0.56).astype(int)
        gt_bndry=np.load(os.path.join(CC3D_BDN_PATH,"bndry","gt",name)+".npy")
        scan_ppl=np.asarray(read_ply_file(os.path.join(CC3D_BDN_PATH,"scan",name)+".ply").points)

        # Model without PPL
        pred_wo_ppl=(np.load(os.path.join(PPL_BDN_PATH,"bndry","pred",name)+".npy")>0.56).astype(int)
        scan_wo_ppl=np.asarray(read_ply_file(os.path.join(PPL_BDN_PATH,"scan",name)+".ply").points)
        pred_wo_ppl=point_label_mapper(scan_wo_ppl,pred_wo_ppl,scan_ppl)
        
        # For boundary
        mainPcd=create_pcd_with_colors(scan_ppl,gt_bndry,[0.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_ppl,[3.0,0.0,0.0])+\
            create_pcd_with_colors(scan_ppl,pred_wo_ppl,[6.0,0.0,0.0]) 

        o3d.io.write_point_cloud(os.path.join(output_path,name.split("/")[-1]+".ply"),mainPcd)
        
    except Exception as e:
        #print(traceback.print_exc())
        #break
        pass


  0%|          | 0/955 [00:00<?, ?it/s]